In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
#TF2
#from tensorflow.python.compiler.tensorrt import trt_convert as trt
#TF1
import tensorflow.contrib.tensorrt as trt
import tensorflow as tf


from tensorflow.python.framework import graph_io
from google.colab import drive
from tensorflow.python.platform import gfile

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [3]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
%cd /content/drive/My\ Drive/ObjectDetection_Retinanet

/content/drive/My Drive/ObjectDetection_Retinanet


In [5]:
output_names = ['filtered_detections/map/TensorArrayStack/TensorArrayGatherV3:0',
                'filtered_detections/map/TensorArrayStack_1/TensorArrayGatherV3:0',
                'filtered_detections/map/TensorArrayStack_2/TensorArrayGatherV3:0']

In [6]:
tensorflow_model_path = 'saved_model/saved_model.pb'

In [7]:
# get frozen graph
def get_frozen_graph(graph_file):
    """Read Frozen Graph file from disk."""
    with tf.gfile.FastGFile(graph_file, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    return graph_def

graph_def = get_frozen_graph(tensorflow_model_path)

Instructions for updating:
Use tf.gfile.GFile.


In [10]:
trt_graph = trt.create_inference_graph(
    input_graph_def=graph_def,
    outputs=output_names,
    max_batch_size=1,
    max_workspace_size_bytes=1 << 25,
    precision_mode='FP16',
    minimum_segment_size=50
)

INFO:tensorflow:Linked TensorRT version: (0, 0, 0)
INFO:tensorflow:Loaded TensorRT version: (0, 0, 0)
INFO:tensorflow:Running against TensorRT version 0.0.0


In [11]:
graph_io.write_graph(trt_graph, "saved_model_small",
                     "trt_graph.pb", as_text=False)

'saved_model_small/trt_graph.pb'